In [50]:
!pip install transformers[torch] sentence-transformers datasets
!pip install -U accelerate
!pip install -U transformers

# Libraries

In [51]:
import torch
import transformers

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import time
import numpy as np
import pandas as pd



# Dataset

In [54]:
from datasets import load_dataset

dataset = load_dataset("imdb")

dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [55]:
small_train_dataset = dataset["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = dataset["test"].shuffle(seed=42).select([i for i in list(range(300))])

# Tokenize

In [56]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


In [57]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [58]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 3000
})

In [59]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

In [60]:
# Select the first 100 rows from tokenized_train to create tokenized_dev
tokenized_dev = tokenized_train.select([i for i in range(100)])

# Print the information about tokenized_dev
print(tokenized_dev)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 100
})


In [74]:
print('Train Dataset Size:', len(tokenized_train))
print('Test Dataset Size:', len(tokenized_test))
print('Development Dataset Size:', len(tokenized_dev))

Train Dataset Size: 3000
Test Dataset Size: 300
Development Dataset Size: 100


In [61]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [62]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


# Baseline Models

## Majority Classifier

In [ ]:
len(tokenized_train_data.input_ids)

29750

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

# Create a majority classifier that always predicts the most frequent class
majority_classifier = DummyClassifier(strategy="most_frequent")
majority_classifier.fit(tokenized_train_data.input_ids, y_train)

# Make predictions on the development set
predictions_majority = majority_classifier.predict(tokenized_dev_data.input_ids)

# Print the classification report for the majority classifier
print(classification_report(y_dev, predictions_majority))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67      2621
           1       0.00      0.00      0.00      2629

    accuracy                           0.50      5250
   macro avg       0.25      0.50      0.33      5250
weighted avg       0.25      0.50      0.33      5250



## Logistic Regression - Lasso

In [ ]:
target_names = [0,1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

best_lambda = 1

clf = LogisticRegression(C=1/best_lambda,penalty='l2')
clf.fit(tokenized_train_data.input_ids, y_train)

predictions = clf.predict(tokenized_dev_data.input_ids)
print(classification_report(y_dev, predictions))

              precision    recall  f1-score   support

           0       0.50      0.55      0.53      2621
           1       0.51      0.46      0.48      2629

    accuracy                           0.51      5250
   macro avg       0.51      0.51      0.50      5250
weighted avg       0.51      0.51      0.50      5250



## MLP

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

mlp_model_path = '/content/drive/My Drive/NLP/Assignment 3/best_stopping_model.h5'

# Load MLP model
mlp_model = load_model(mlp_model_path)
mlp_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 416)               208416    
                                                                 
 dropout_4 (Dropout)         (None, 416)               0         
                                                                 
 dense_7 (Dense)             (None, 32)                13344     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 221793 (866.38 KB)
Trainable params: 221793 (866.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

mlp_model = Sequential([
    Dense(416, activation='relu', input_shape=(250,)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

mlp_model.compile(optimizer=Adam(lr=0.002), loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
# mlp_model.fit(tokenized_train_data.input_ids, y_train, epochs=70, batch_size=256, validation_data=(tokenized_dev_data.input_ids, y_dev))
mlp_model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 416)               104416    
                                                                 
 dropout_4 (Dropout)         (None, 416)               0         
                                                                 
 dense_7 (Dense)             (None, 32)                13344     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 117793 (460.13 KB)
Trainable params: 117793 (460.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Make predictions on dev data using the trained MLP model
predictions = (mlp_model.predict(tokenized_dev_data.input_ids))
predictions = (mlp_model.predict(tokenized_dev_data.input_ids) > 0.5).astype(int)

# Print classification report
print(classification_report(y_dev, predictions))

## Bi-RNNs - Linear Attention

In [ ]:
birnn_model_path = '/content/drive/My Drive/NLP/Assignment4/best_model.h5'
# Load BiRNN model
birnn_model = load_model(birnn_model_path)
birnn_model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

birnn_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 300)          6000600   
                                                                 
 bidirectional_2 (Bidirecti  (None, 256)               330240    
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6347353 (24.21 MB)
Trainable params: 346753 (1.32 MB)
Non-trainable params: 6000600 (22.89 MB)
_________________________________________________________________


In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Assuming dev_data has shape (None, 250)
padded_dev_data = pad_sequences(tokenized_dev_data.input_ids, maxlen=500, padding='post')

# Evaluate the model on the padded validation data
loss, accuracy = birnn_model.evaluate(padded_dev_data, y_dev)

print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')


165/165 [==============================] - 94s 538ms/step - loss: 1.0651 - accuracy: 0.5198
Validation Loss: 1.0651, Validation Accuracy: 0.5198


In [ ]:
# Make predictions on dev data using the trained BiRNN model
predictions = (birnn_model.predict(padded_dev_data) > 0.5).astype(int)

# Print classification report
print(classification_report(y_dev, predictions))

165/165 [==============================] - 62s 375ms/step
              precision    recall  f1-score   support

           0       0.51      0.81      0.63      2621
           1       0.55      0.23      0.32      2629

    accuracy                           0.52      5250
   macro avg       0.53      0.52      0.47      5250
weighted avg       0.53      0.52      0.47      5250



## CNNs

In [ ]:
cnn_model_path = '/content/drive/My Drive/NLP/Assignment5/cnn_best_stopping_model.h5'
# Load BiRNN model
cnn_model = load_model(cnn_model_path)
cnn_model.compile(optimizer=Adam(lr=1.0), loss='binary_crossentropy', metrics=['accuracy'])

cnn_model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 250)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 250, 300)             3000000   ['input_3[0][0]']             
                                                          0                                       
                                                                                                  
 dropout_6 (Dropout)         (None, 250, 300)             0         ['embedding_2[0][0]']         
                                                                                                  
 2-gram_Convolutions (Conv1  (None, 250, 128)             76928     ['dropout_6[0][0]']     

In [ ]:
# Make predictions using the loaded CNN model
predictions = cnn_model.predict(pad_sequences(tokenized_dev_data.input_ids, maxlen=250, padding='post'))

# If the predictions are probabilities and you want to threshold them:
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)


165/165 [==============================] - 1s 4ms/step


In [ ]:
# Convert binary predictions and ground truth labels to 1D arrays
binary_predictions_flat = binary_predictions.flatten()

# Print classification report
print(classification_report(y_dev, binary_predictions_flat))

              precision    recall  f1-score   support

           0       0.50      0.08      0.13      2621
           1       0.50      0.92      0.65      2629

    accuracy                           0.50      5250
   macro avg       0.50      0.50      0.39      5250
weighted avg       0.50      0.50      0.39      5250



# Training

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  #  output_dir=repo_name,
   output_dir='./distilBert_sentiment_classifier/',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True
  #  push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.306445,0.873333,0.880503
2,No log,0.367816,0.886667,0.891720
3,0.244800,0.403952,0.866667,0.873418


<ipython-input-11-2457e8bb4065>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

TrainOutput(global_step=564, training_loss=0.2291839283408848, metrics={'train_runtime': 439.3527, 'train_samples_per_second': 20.485, 'train_steps_per_second': 1.284, 'total_flos': 1176910742965440.0, 'train_loss': 0.2291839283408848, 'epoch': 3.0})

In [ ]:
# # Save the model to Google Drive
# trainer.save_model("/content/drive/MyDrive/NLP/Assignment6/distilBert")

In [16]:
from transformers import AutoModelForSequenceClassification

# Load the pre-trained model
loaded_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/NLP/Assignment6/distilBert")
loaded_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# Evaluation

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.3064454197883606,
 'eval_accuracy': 0.8733333333333333,
 'eval_f1': 0.8805031446540881,
 'eval_runtime': 5.837,
 'eval_samples_per_second': 51.396,
 'eval_steps_per_second': 3.255,
 'epoch': 3.0}

# Hyperparameter Tuning

In [17]:
model = loaded_model

In [20]:
from transformers import TrainingArguments, Trainer

# Define hyperparameters for tuning
learning_rates = [4e-5, 3e-5, 2e-5]
batch_sizes = [8, 16]
weight_decays = [0.1, 0.01, 0.001]
num_epochs_list = [2, 3]

best_accuracy = 0
best_hyperparameters = {}

# Loop over hyperparameters
for lr in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
            for num_epochs in num_epochs_list:
                # Update hyperparameters in TrainingArguments
                training_args.learning_rate = lr
                training_args.per_device_train_batch_size = batch_size
                training_args.weight_decay = weight_decay
                training_args.num_train_epochs = num_epochs

                # Create Trainer with updated hyperparameters
                trainer = Trainer(
                    model=model,
                    args=training_args,
                    train_dataset=tokenized_train,
                    eval_dataset=tokenized_dev,
                    tokenizer=tokenizer,
                    data_collator=data_collator,
                    compute_metrics=compute_metrics,
                )

                # Train the model
                trainer.train()

                # Evaluate on the development set
                results = trainer.evaluate()
                accuracy = results['eval_accuracy']

                print(f'LR: {lr}, Batch Size: {batch_size}, Weight Decay: {weight_decay}, Epochs: {num_epochs}, Accuracy: {accuracy}')

                # Update best hyperparameters if needed
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_hyperparameters['learning_rate'] = lr
                    best_hyperparameters['per_device_train_batch_size'] = batch_size
                    best_hyperparameters['weight_decay'] = weight_decay
                    best_hyperparameters['num_epochs'] = num_epochs

# Train the best model on the full training set with the best hyperparameters
training_args.learning_rate = best_hyperparameters['learning_rate']
training_args.per_device_train_batch_size = best_hyperparameters['per_device_train_batch_size']
training_args.weight_decay = best_hyperparameters['weight_decay']
training_args.num_train_epochs = best_hyperparameters['num_epochs']

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.000052,1.000000,1.000000
2,0.089600,0.000013,1.000000,1.000000


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

LR: 4e-05, Batch Size: 8, Weight Decay: 0.1, Epochs: 2, Accuracy: 1.0


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.033625,0.990000,0.989247
2,0.080500,0.000010,1.000000,1.000000
3,0.020800,0.000004,1.000000,1.000000


...

LR: 2e-05, Batch Size: 16, Weight Decay: 0.001, Epochs: 3, Accuracy: 1.0


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.000000,1.000000,1.000000
2,0.000000,0.000000,1.000000,1.000000


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

TrainOutput(global_step=750, training_loss=0.0, metrics={'train_runtime': 283.7788, 'train_samples_per_second': 21.143, 'train_steps_per_second': 2.643, 'total_flos': 744373639307040.0, 'train_loss': 0.0, 'epoch': 2.0})

In [39]:
best_hyperparameters_dict = {
    'learning_rate': best_hyperparameters['learning_rate'],
    'per_device_train_batch_size': best_hyperparameters['per_device_train_batch_size'],
    'weight_decay': best_hyperparameters['weight_decay'],
    'num_epochs': best_hyperparameters['num_epochs'],
}

In [40]:
best_hyperparameters_dict

{'learning_rate': 4e-05,
 'per_device_train_batch_size': 8,
 'weight_decay': 0.1,
 'num_epochs': 2}

In [41]:
import json

# Save the best hyperparameters to a JSON file
output_file_path = '/content/drive/My Drive/NLP/Assignment6/best_hyperparameters.json'  # Adjust the path as needed
with open(output_file_path, 'w') as output_file:
    json.dump(best_hyperparameters_dict, output_file)

print(f"Best hyperparameters saved to: {output_file_path}")

Best hyperparameters saved to: /content/drive/My Drive/NLP/Assignment6/best_hyperparameters.json


In [21]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

In [ ]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#   #  output_dir=repo_name,
#    output_dir='./distilBert_sentiment_classifier/',
#    learning_rate=4e-05,
#    per_device_train_batch_size=8,
#    per_device_eval_batch_size=8,
#    num_train_epochs=2,
#    weight_decay=0.1,
#    evaluation_strategy="epoch",
#    save_strategy="epoch",
#    load_best_model_at_end=True
#   #  push_to_hub=True,
# )

# trainer = Trainer(
#    model=model,
#    args=training_args,
#    train_dataset=tokenized_train,
#    eval_dataset=tokenized_test,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )

# trainer.train()


# Evaluation

In [33]:
trainer.evaluate(eval_dataset=tokenized_test)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 2.6522586345672607,
 'eval_accuracy': 0.8633333333333333,
 'eval_f1': 0.8690095846645367,
 'eval_runtime': 6.7309,
 'eval_samples_per_second': 44.571,
 'eval_steps_per_second': 2.823,
 'epoch': 2.0}

In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    preds = (predictions > 0.5).astype(int)

    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds),
        'recall': recall_score(labels, preds),
        'f1': f1_score(labels, preds),
        'roc_auc': roc_auc_score(labels, predictions),
    }


In [68]:
results = trainer.evaluate(eval_dataset=tokenized_test)
results


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.3693791329860687,
 'eval_accuracy': 0.8666666666666667,
 'eval_f1': 0.8639455782312925,
 'eval_runtime': 5.6233,
 'eval_samples_per_second': 53.349,
 'eval_steps_per_second': 6.758,
 'epoch': 2.0}

In [69]:
# Extract specific metrics
accuracy = results['eval_accuracy']
f1 = results['eval_f1']
loss = results['eval_loss']

# Print individual metrics
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Loss: {loss}")

Accuracy: 0.8666666666666667
F1 Score: 0.8639455782312925
Loss: 0.3693791329860687


In [80]:
# Evaluate on the validation set
val_results = trainer.evaluate(eval_dataset=tokenized_dev)
print("Validation results")
print('---------------------')
val_results


# Evaluate on the test set
test_results = trainer.evaluate(eval_dataset=tokenized_test)
print("Test results")
print('---------------------')
test_results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

Validation results
---------------------


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Test results
---------------------


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 0.3693791329860687,
 'eval_accuracy': 0.8666666666666667,
 'eval_f1': 0.8639455782312925,
 'eval_runtime': 5.912,
 'eval_samples_per_second': 50.744,
 'eval_steps_per_second': 6.428,
 'epoch': 2.0}

# Performance Comparison Across Models

|Model Name|Val Accuracy|Test Accuracy|
|----------|--------|--|
|Majority Classifier + TF-IDF|49.24%|49.71% |
|Logistic Regression + TF-IDF + L1|87.22%|87.87% |
|MLP + TF-IDF + L1|87.94%|       88.10%    |
|RNN Word2Vec + self-attention|85.62%|85.87%|
|CNN|86.89%|86.13%|
|**Transformers**|**100%**|**86.33%**|
